In [1]:
#########################################################
# Import Several Packages
#########################################################
from math import *
from tqdm import tqdm
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from matplotlib.animation import FuncAnimation
import seaborn as sns
import warnings
import json
import sys
import queue

warnings.filterwarnings(action='ignore')


# Set Random Seed
np.random.seed(0)


#########################################################
# 구현해야 하는 기본 모듈: map / patient / hospital

#########################################################
######################## For MAP ########################
#########################################################
"""
[ Attributes] 

1. GRID:
	- min_lat								(float)
    - max_lat								(float)
    - min_lon								(float)
    - max_lon								(float)
2. districts: 								(dict)
	{0:[center_location, radius], ...}
    - 0: 관악구
    - center_location: mu_1, mu_2 			(tuple)
    - radius: np.sqrt(size/np.pi) 			(float)
3. weights: 								(list)
	[관악구 인구 %, 그 외 다른 거 %, ]
4. patient_dict:							(dict)
	{id: class Patient(), ...}
5. acc_patient_num:							(int)

[ Methods ]

1. create_patient(self, no arg)				(func)
	- id = self.acc_patient_num
    - d_id = np.random.choice(self.weights)
      lat, lon = np.random.multivariatenormal(self.districts[d_id][0], self.districts[d_id][0])
      lat = max(min(lat, max_lat), min_lat)
      lon = max(min(lon, max_lon), min_lon)
      location = (lat, lon)
    - created_time = tau
    - state = 0
    - destiny = np.random."oracle"
      (oracle: 처음에는 사망 확률이 낮다가, 시간 지나면 상승, left skewed)			[GPT 부탁해!]
    
    (그래서)
    self.patient_dict[id] = Patient(id, location, created_time, state, destiny)
    
    return
"""

'\n[ Attributes] \n\n1. GRID:\n\t- min_lat\t\t\t\t\t\t\t\t(float)\n    - max_lat\t\t\t\t\t\t\t\t(float)\n    - min_lon\t\t\t\t\t\t\t\t(float)\n    - max_lon\t\t\t\t\t\t\t\t(float)\n2. districts: \t\t\t\t\t\t\t\t(dict)\n\t{0:[center_location, radius], ...}\n    - 0: 관악구\n    - center_location: mu_1, mu_2 \t\t\t(tuple)\n    - radius: np.sqrt(size/np.pi) \t\t\t(float)\n3. weights: \t\t\t\t\t\t\t\t(list)\n\t[관악구 인구 %, 그 외 다른 거 %, ]\n4. patient_dict:\t\t\t\t\t\t\t(dict)\n\t{id: class Patient(), ...}\n5. acc_patient_num:\t\t\t\t\t\t\t(int)\n\n[ Methods ]\n\n1. create_patient(self, no arg)\t\t\t\t(func)\n\t- id = self.acc_patient_num\n    - d_id = np.random.choice(self.weights)\n      lat, lon = np.random.multivariatenormal(self.districts[d_id][0], self.districts[d_id][0])\n      lat = max(min(lat, max_lat), min_lat)\n      lon = max(min(lon, max_lon), min_lon)\n      location = (lat, lon)\n    - created_time = tau\n    - state = 0\n    - destiny = np.random."oracle"\n      (oracle: 처음에는 사망 확

In [2]:
# Define Classes: Patient, Hospital

class Patient():
  def __init__(self, id:int, location:tuple, created_time:int, destiny:float):
    self.id = id
    self.location = location   #tuple (float, float)
    self.created_time = created_time
    self.destiny = destiny # 사망 시간
    self.decision = None # Optimizer에서 결정되어 할당 (향하고 있는 병원)
    self.treatment_time = None # Optimizer에서 결정되어 할당 (치료 시간)

    
class Hospital():
  def __init__(self, hospitals_id:int, hospitals_location:tuple, mean_treat_time:int):
    self.queue = [] # queue.Queue()
    self.len_queue = 0
    self.treat_start = None
    self.id = hospitals_id
    self.location = hospitals_location
    self.mean_treat_time = mean_treat_time
  
  def sample_treat_time(self):
    treat_time = np.random.exponential(self.mean_treat_time, 1)
    return treat_time

In [5]:
def create_patient(model_type: str):
    """
    model_type: 'naive' or 'opt'
    """
    global tau, acc_patient_num, weights, districts, GRID 
    
    id = acc_patient_num
    d_id = np.random.choice(len(weights), p=weights)
    mean = districts[d_id][0]
    cov = [[districts[d_id][1]**2, 0], [0, districts[d_id][1]**2]]
    
    lon, lat = np.random.multivariate_normal(mean, cov)
    lat = max(min(lat, GRID['max_lat']), GRID['min_lat'])
    lon = max(min(lon, GRID['max_lon']), GRID['min_lon'])
    location = (lon, lat)

    created_time = tau # Global time step
    
    alpha_skewed = 8 # Standard Alpha
    beta_skewed = 3 # Standard Beta 
    a_skewed = 0 # Min 생존 시간
    b_skewed = 3600 # Max 생존 시간
    
    # skew 고려해서 standard beta 설정 (by alpha_skewed, beta_skewed)    
    destiny_temp = np.random.beta(alpha_skewed, beta_skewed, 1)
                   
    # domain 늘이기 (from a_skewed to b_skewed)
    destiny = a_skewed + (b_skewed - a_skewed) * destiny_temp

    print("환자 {}가 생성되었습니다. 위치: {}, 생성 시간: {}, 사망 시간: {}".format(id, location, created_time, destiny))

    # Assuming Patient class is defined elsewhere
    patient_dict[id] = Patient(id, location, created_time, destiny)
    
    acc_patient_num += 1

    # Determine the hospital that the patient should go using optimizer
    if model_type == 'opt':
      patient_dict[id].decision = optimizer(id)
    elif model_type == 'naive':
      patient_dict[id].decision = naive_optimizer(id)
    else:
      raise ValueError("model_type should be either 'opt' or 'naive'")

def compute_etaw(patient: Patient, hospital: Hospital, tau: int):
  patient_lon, patient_lat = patient.location
  hospital_lon, hospital_lat = hospital.location

  # Calculate the real distance using l2 norm
  distance = np.sqrt((patient_lat - hospital_lat)**2 + (patient_lon - hospital_lon)**2)  ## km
  eta = distance / 120 * 3600  
  wait = (hospital.len_queue + 1 + np.sqrt(hospital.len_queue + 1)) * hospital.mean_treat_time
  etaw = eta + wait
    
  return eta, etaw

def get_new_location(patient_location, hospital_location):
  patient_lon, patient_lat = patient_location
  hospital_lon, hospital_lat = hospital_location

  # Calculate the real distance using l2 norm
  distance = np.sqrt((patient_lat - hospital_lat)**2 + (patient_lon - hospital_lon)**2)  ## km
  print("distance: {}".format(distance))
  distance_sec = 120 / 3600
  new_patient_lat = (patient_lat * (distance - distance_sec) + hospital_lat * distance_sec) / distance
  new_patient_lon = (patient_lon * (distance - distance_sec) + hospital_lon * distance_sec) / distance
  
  return (new_patient_lon, new_patient_lat)
  


  
def updater(model_type: str):  
  global patient_dict, hospitals_dict, tau, death_toll
  """
  환자의 현재 위치 업데이트 (V)
  환자가 향하는 병원 업데이트 (V)
  사망 여부 확인 업데이트 (V)
  병원 도착 여부 확인 업데이트 (V)
  각 병원의 queue 업데이트 (V)
  각 병원의 치료 완료한 환자 수 업데이트 (V)  
  """

  """
  model_type: 'naive' or 'opt'
  """
  print("[step 1, 2] 환자의 현재 위치 & 사망 여부 업데이트")

  death_list = []
  arrival_list = []

  for id in patient_dict.keys():
    #################################################################
    ############# Step 1: 환자가 향해야 하는 병원 업데이트 #############
    # 처음에 환자가 생성되면, 해당 환자가 향해야 하는 병원을 optimizer를 통해 결정 (앞에서)
    # 여기서는 모든 환자가 1분마다 업데이트를 통해, 환자가 향해야 하는 병원을 다시 결정

    patient_dict[id].location = get_new_location(patient_dict[id].location, patient_dict[id].decision.location) # 이동하고 있는 환자의 위치를 새로이 정의

    if model_type == 'opt':
      if tau % 60 == 0: # 1분마다 업데이트 (의사결정 주기)
        curr_opt_hospital = optimizer(id) # 현재 환자가 향해야 하는 병원을 새로이 정의
        if patient_dict[id].decision != curr_opt_hospital:
          patient_dict[id].decision = curr_opt_hospital
          print("환자 {}의 이동 경로가 변경되었습니다. from {} to {}".format(id, patient_dict[id].decision.id, curr_opt_hospital.id))

    #################################################################
    ################# Step 2: 사망 여부 확인 업데이트 #################
    # 사망 여부 확인
    if tau - patient_dict[id].created_time > patient_dict[id].destiny:
      print("환자 {}가 사망했습니다.".format(id))
      death_list.append(id)
      # 사망자 수 업데이트
      death_toll += 1
      continue

    #################################################################
    ############# Step 3: 병원 도착 여부 확인 업데이트 #############

    # 모든 환자가 각자 향하는 병원에 도착했는지 확인
    patient_lon, patient_lat = patient_dict[id].location
    if id == 0:
      print("환자 {}의 현재 위치: {}".format(id, patient_dict[0].location))
    hospital_lon, hospital_lat = patient_dict[id].decision.location
    # 평면좌표계로 변환했으니, 0.1km = 100m 이내 도착시 도착으로 간주
    distance = np.sqrt((patient_lat - hospital_lat)**2 + (patient_lon - hospital_lon)**2)
    if distance < 0.1:
      # 환자의 treatment_time을 결정
      patient_dict[id].treatment_time = patient_dict[id].decision.sample_treat_time()      
      # 환자가 도착했다면, 환자를 patient_dict에서 삭제
      # 그 instance를 병원의 queue에 넣고, 병원의 len_queue를 1 증가
      patient_dict[id].decision.queue.append(patient_dict[id])
      patient_dict[id].decision.len_queue += 1
      print("환자 {}가 병원 {}에 도착했습니다.".format(id, patient_dict[id].decision.id))
      print("환자 {}가 병원 {}의 {}번째 환자가 되었습니다.".format(id, patient_dict[id].decision.id, patient_dict[id].decision.len_queue))
      arrival_list.append(id)

  # 사망자 삭제
  for id in death_list:
    del patient_dict[id]

  # 도착자 삭제
  for id in arrival_list:
    del patient_dict[id]
  
  print("[step 3, 4] 각 병원의 queue & 치료 완료한 환자 수 업데이트")


  #################################################################
  ################# Step 4: 각 병원의 queue 업데이트 #################
  for key in hospitals_dict.keys():

    death_hospital_list = []
    treatment_hospital_list = []

    for i, patient in enumerate(hospitals_dict[key].queue[1:]):
      # 각 병원의 대기열 queue에 있는 환자들의 사망 여부 확인
      if tau - patient.created_time > patient.destiny:
        print("환자 {}가 대기중인 병원 {}에서 사망했습니다.".format(patient.id, key))
        death_hospital_list.append(patient.id)
        # 사망자 수 업데이트
        death_toll += 1
        # 병원의 len_queue를 1 감소
        hospitals_dict[key].len_queue -= 1
        continue
    # 각 병원의 대기열 queue에 있는 환자들의 치료 완료 여부 확인
    if hospitals_dict[key].len_queue > 0:
      if tau - hospitals_dict[key].treat_start > hospitals_dict[key].queue[0].treatment_time:
        print("환자 {}가 병원 {}에서 치료를 완료했습니다.".format(hospitals_dict[key].queue[0].id, key))
        # treatment_toll 업데이트
        treatment_toll += 1
        # etaw_toll 업데이트
        etaw_toll += tau - hospitals_dict[key].queue[0].created_time
        treatment_hospital_list.append(key)
        # 병원의 len_queue를 1 감소
        hospitals_dict[key].len_queue -= 1

      if hospitals_dict[key].len_queue > 0:
        hospitals_dict[key].treat_start = tau
      else:
        hospitals_dict[key].treat_start = None

    # 사망자 삭제
    for id in death_hospital_list:
      del hospitals_dict[key].queue[id]

    # 치료 완료자 삭제
    for id in treatment_hospital_list:
      del hospitals_dict[key].queue[id]


def naive_optimizer(patient_id):
  """
  Just return the hospital that is closest to the patient
  Input: patient_id
  Output: hospital_id (closest)
  """
  global patient_dict, hospitals_dict, tau
  
  patient = patient_dict[patient_id]

  min_distance = sys.maxsize
  min_hospital = None

  for hospital_id in hospitals_dict.keys():
    # Calculate the distance using l2 norm
    distance = np.sqrt((patient.location[0] - hospitals_dict[hospital_id].location[0])**2 + (patient.location[1] - hospitals_dict[hospital_id].location[1])**2)
    if distance < min_distance:
      min_distance = distance
      min_hospital = hospitals_dict[hospital_id]

  # Return min_hospital
  return min_hospital


def optimizer(patient_id):
  """
  Return the hospital that minimizes the ETAW for each patient
  Input: patient_id
  Output: hospital_id (minimize ETAW)
  """
  global patient_dict, hospitals_dict, tau
  
  patient = patient_dict[patient_id]

  min_etaw = sys.maxsize
  min_hospital = None

  for hospital_id in hospitals_dict.keys():
    eta, etaw = compute_etaw(patient, hospitals_dict[hospital_id], tau)
    if etaw < min_etaw:
      min_etaw = etaw
      min_hospital = hospitals_dict[hospital_id]

  # Return min_hospital
  return min_hospital

In [6]:
with open('./config.json') as f:
    config = json.load(f)
    
model_type = input("Enter the model type: (opt or naive) ")

min_lat = config['min_lat']
max_lat = config['max_lat']
min_lon = config['min_lon']
max_lon = config['max_lon']

districts = config['districts'] # dict; {0: [(lon, lat), radius], ...}
# Transform the key of districts from str to int
districts = {int(key): districts[key] for key in districts.keys()}
weights = config['weights'] # list; population percentages
hospitals = config['hospitals'] # dict; {0: (lon, lat), ...}
# Transform the key of hospitals from str to int
hospitals = {int(key): hospitals[key] for key in hospitals.keys()}
mean_treat_times = config['mean_treat_times'] # list; average treatment time in each hospital
    
GRID = {'min_lat': min_lat, 'max_lat': max_lat, 'min_lon': min_lon, 'max_lon': max_lon}
patient_dict = {}  # dict; To store Patient instances
acc_patient_num = 0 # int; accumulated patient number

tau = 0 # Simulation global time step 
end_time = 24 * 3600

# Initialize the time points when patients occur on the map
patient_generate_mean = 1200

patient_generate_points = []
time_point = 0

while time_point < end_time:
    sample = np.random.exponential(1200, 1).astype(int)[0]
    time_point += sample
    
    if time_point < end_time:
        patient_generate_points.append(time_point)

treatment_toll = 0
death_toll = 0
etaw_toll = 0
patient_toll = len(patient_generate_points)

hospitals_dict = {}
for id in range(len(hospitals)):
    hospitals_dict[id] = Hospital(id, hospitals[id], mean_treat_times[id])
    
metrics_df = pd.DataFrame(columns=['treatment_toll', 'death_toll', 'etaw_toll', 'patient_toll'])


def draw_cur_map():
    global tau, patient_dict, hospitals, min_lat, max_lat, min_lon, max_lon

    # Plot the current map
    plt.figure(figsize=(10, 10))
    m = Basemap(projection='merc', llcrnrlat=min_lat, urcrnrlat=max_lat, llcrnrlon=min_lon, urcrnrlon=max_lon, resolution='i')
    m.drawcoastlines()
    m.drawcountries()
    m.drawmapboundary(fill_color='white')
    m.fillcontinents(color='white', lake_color='white')
    m.drawparallels(np.arange(min_lat, max_lat, 0.1), labels=[1, 0, 0, 0])
    m.drawmeridians(np.arange(min_lon, max_lon, 0.1), labels=[0, 0, 0, 1])
    # Plot the hospitals
    for id in range(len(hospitals)):
        x, y = m(hospitals[id][0], hospitals[id][1])
        m.plot(x, y, 'bo', markersize=5)
    # Plot the patients
    for id in patient_dict.keys():
        x, y = m(patient_dict[id].location[0], patient_dict[id].location[1])
        m.plot(x, y, 'ro', markersize=5)
    plt.title("Time: {}".format(tau))
    plt.savefig('./images/{}.png'.format(tau))    


for tau in tqdm(range(0, end_time)):  # start_time should be defined, or use 0 if it starts from the beginning
    if tau in patient_generate_points:
        print("Time: {}".format(tau))
        create_patient(model_type)  # Create patient
        print("환자 {}가 생성되었습니다.".format(acc_patient_num - 1))
        updater(model_type)  # Update patient's location + alpha
        print("환자 {}의 위치가 업데이트되었습니다.".format(acc_patient_num - 1))
    
    # Add the current metrics to the metrics_df
    metrics_df.loc[tau] = [treatment_toll, death_toll, etaw_toll, patient_toll]


# Save the metrics_df
metrics_df.to_csv('./metrics_df.csv')

  0%|          | 192/86400 [00:00<01:26, 991.64it/s]

Time: 24
환자 0가 생성되었습니다. 위치: (11233.895671946328, 3556.7973921833695), 생성 시간: 24, 사망 시간: [3172.17625934]
환자 0가 생성되었습니다.
[step 1, 2] 환자의 현재 위치 & 사망 여부 업데이트
distance: 6.175926959577677
환자 0의 현재 위치: (11233.887342748245, 3556.797129688133)
[step 3, 4] 각 병원의 queue & 치료 완료한 환자 수 업데이트
환자 0의 위치가 업데이트되었습니다.


  3%|▎         | 2211/86400 [00:03<02:30, 561.13it/s]

Time: 2142
환자 1가 생성되었습니다. 위치: (11221.98891375696, 3562.1821741347435), 생성 시간: 2142, 사망 시간: [2880.8271629]
환자 1가 생성되었습니다.
[step 1, 2] 환자의 현재 위치 & 사망 여부 업데이트
distance: 6.167593626243981
환자 0의 현재 위치: (11233.879013550162, 3556.796867192896)
distance: 3.1079677225329223
[step 3, 4] 각 병원의 queue & 치료 완료한 환자 수 업데이트
환자 1의 위치가 업데이트되었습니다.
Time: 2147
환자 2가 생성되었습니다. 위치: (11211.135388793364, 3570.363622992294), 생성 시간: 2147, 사망 시간: [1939.49563028]
환자 2가 생성되었습니다.
[step 1, 2] 환자의 현재 위치 & 사망 여부 업데이트
distance: 6.159260292910286
환자 0의 현재 위치: (11233.87068435208, 3556.7966046976594)
distance: 3.0996343891992026
distance: 7.814446723558334
[step 3, 4] 각 병원의 queue & 치료 완료한 환자 수 업데이트
환자 2의 위치가 업데이트되었습니다.


  4%|▍         | 3569/86400 [00:06<04:03, 339.47it/s]

...

Time: 24285
환자 18가 생성되었습니다. 위치: (11231.813344406271, 3561.0749816199204), 생성 시간: 24285, 사망 시간: [1876.71169183]
환자 18가 생성되었습니다.
[step 1, 2] 환자의 현재 위치 & 사망 여부 업데이트
distance: 8.98433385986756
환자 15가 사망했습니다.
distance: 7.556070701352353
distance: 3.2721958516284726
distance: 5.539283779173379
[step 3, 4] 각 병원의 queue & 치료 완료한 환자 수 업데이트
환자 18의 위치가 업데이트되었습니다.


 31%|███       | 26913/86400 [00:44<01:38, 604.88it/s]

Time: 26840
환자 19가 생성되었습니다. 위치: (11208.595649404686, 3559.020960504836), 생성 시간: 26840, 사망 시간: [2558.00952361]
환자 19가 생성되었습니다.
[step 1, 2] 환자의 현재 위치 & 사망 여부 업데이트
distance: 7.547737368018741
환자 16가 사망했습니다.
distance: 3.2638625182958423
환자 17가 사망했습니다.
distance: 5.5309504458398395
환자 18가 사망했습니다.
distance: 2.3945214043785437
[step 3, 4] 각 병원의 queue & 치료 완료한 환자 수 업데이트
환자 19의 위치가 업데이트되었습니다.


 32%|███▏      | 27661/86400 [00:46<01:37, 600.84it/s]


KeyboardInterrupt: 

,treatment_toll,death_toll,etaw_toll,patient_toll
1,0,0,0,81
2,0,0,0,81
3,0,0,0,81
4,0,0,0,81
5,0,0,0,81
...,...,...,...,...
86397,0,79,0,81
86398,0,79,0,81
86399,0,79,0,81
86400,0,79,0,81


In [ ]:
patient_generate_points

[22,
 1916,
 2220,
 2728,
 5886,
 7348,
 7386,
 7601,
 8766,
 9799,
 10125,
 13390,
 14532,
 15452,
 16521,
 18092,
 18540,
 19149,
 19431,
 19678,
 23144,
 24758,
 25567,
 25876,
 26228,
 26299,
 26982,
 27430,
 28860,
 29429,
 29666,
 29695,
 29778,
 31143,
 31868,
 32790,
 35513,
 41080,
 41373,
 42678,
 43044,
 43069,
 44773,
 45235,
 45815,
 46880,
 49013,
 50202,
 52674,
 53057,
 54976,
 55222,
 58885,
 60280,
 60571,
 64104,
 65679,
 66030,
 66317,
 67193,
 67224,
 67503,
 68166,
 68728,
 69475,
 69865,
 70925,
 73317,
 73467,
 74341,
 74510,
 76024,
 76629,
 77629,
 77871,
 78058,
 78861,
 79388,
 82772,
 84511,
 86168]